# Project Orange: IMDB reviews sentiment analysis

In [366]:
# Install a pip package in the current Jupyter kernel
# This is better than !pip ... see https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install spacy pandas

In [367]:
import pandas as pd
import re
import spacy
from spacy.lang.en import English

### Import dataset

In [368]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("IMDB Dataset.csv") 

# Keep the first 10 elements to reduce the load on cpu
data=data[:10]
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Text to lowercase

In [369]:
def to_lower(this_review):
    this_review=this_review.lower()
    return this_review
    
data['review'] = data['review'].map(to_lower)
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


### Remove HTML elements

In [370]:
REMOVE_HTML = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def remove_html(review):
    return REMOVE_HTML.sub("", review) 

data['cleaned_review'] = data['review'].map(remove_html)
data.head()

,review,sentiment,cleaned_review
0,one of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,a wonderful little production. <br /><br />the...,positive,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love in the time of money"" is..."


### Identify and remove entities

In [371]:
from spacy import displacy

def recognize_it(this_review):

    doc=sp(this_review)

  
    for i in doc.ents:
            i=str(i)
            this_review=this_review.replace(" "+i,"")
    return this_review


In [372]:
data['IDcleaned_review'] = data['cleaned_review'].map(recognize_it)
data.head()

,review,sentiment,cleaned_review,IDcleaned_review
0,one of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...
1,a wonderful little production. <br /><br />the...,positive,a wonderful little production. the filming tec...,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love in the time of money"" is...","petter's ""love in the time of money"" is a visu..."


### Lemmatization

In [373]:
sp= spacy.load('en_core_web_sm')
# Implementing lemmatization
def lemmatize_it(this_review):
    filtered_sent=[]

    #  "nlp" Object is used to create documents with linguistic annotations.
    lem = sp(this_review)
    
   # finding lemma for each word
    for word in lem:
        filtered_sent.append(word.lemma_)
    return filtered_sent

In [374]:
data['lemmatized_review'] = data['IDcleaned_review'].map(lemmatize_it)
data.head()

,review,sentiment,cleaned_review,IDcleaned_review,lemmatized_review
0,one of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...,"[one, of, the, other, reviewer, have, mention,..."
1,a wonderful little production. <br /><br />the...,positive,a wonderful little production. the filming tec...,a wonderful little production. the filming tec...,"[a, wonderful, little, production, ., the, fil..."
2,i thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,i thought this was a wonderful way to spend ti...,"[i, think, this, be, a, wonderful, way, to, sp..."
3,basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,basically there's a family where a little boy ...,"[basically, there, be, a, family, where, a, li..."
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love in the time of money"" is...","petter's ""love in the time of money"" is a visu...","[petter, 's, "", love, in, the, time, of, money..."


### Tokenization (not used)

In [375]:
# "nlp" Object is used to create documents with linguistic annotations.
nlp = English()

def tokenize_review(this_review):
    my_doc = nlp(this_review)
    
    # Create list of word tokens
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    return token_list

In [376]:
#data['tokenized_review'] = data['cleaned_review'].map(tokenize_review)
#data.head()

### Adapt spacy stopwords list to our topic

In [382]:
#print stopword list from spacy
spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)

remove_from_stopwordlist=["n't", "most", "much", "never", "no", "not", "nothing", "n‘t", "n’t", "really", "top", "very", "well"]
for word in spacy_stopwords:
    if word in remove_from_stopwordlist:
         spacy_stopwords.remove(word)

add_to_stopwords=['.', ',', '!', '?', ':', '&', '...', '(', ')','-', '/', '"', ';']
for word in add_to_stopwords:
    spacy_stopwords.append(word)

###  Remove stopwords and punctuation

In [378]:
def eliminate_stopwords(this_review):
    
    filtered_sent=[]

    #  "nlp" Object is used to create documents with linguistic annotations.
    doc = this_review
    
    # filtering stop words
    for word in doc:
        if word not in spacy_stopwords:
            filtered_sent.append(word)
    return filtered_sent
    

In [379]:
data['stopcleaned_review'] = data['lemmatized_review'].map(eliminate_stopwords)
data.head()

,review,sentiment,cleaned_review,IDcleaned_review,lemmatized_review,stopcleaned_review
0,one of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,one of the other reviewers has mentioned that ...,"[one, of, the, other, reviewer, have, mention,...","[reviewer, mention, watch, oz, episode, -PRON-..."
1,a wonderful little production. <br /><br />the...,positive,a wonderful little production. the filming tec...,a wonderful little production. the filming tec...,"[a, wonderful, little, production, ., the, fil...","[wonderful, little, production, filming, techn..."
2,i thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,i thought this was a wonderful way to spend ti...,"[i, think, this, be, a, wonderful, way, to, sp...","[think, wonderful, way, spend, time, hot, sit,..."
3,basically there's a family where a little boy ...,negative,basically there's a family where a little boy ...,basically there's a family where a little boy ...,"[basically, there, be, a, family, where, a, li...","[basically, family, little, boy, jake, think, ..."
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love in the time of money"" is...","petter's ""love in the time of money"" is a visu...","[petter, 's, "", love, in, the, time, of, money...","[petter, love, time, money, visually, stunning..."


Do we do Bag of Words?
Do we do TF-IDF?

### Splitting the dataset into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X = data['stopcleaned_review'] # the features we want to analyze, we can play with others too
y = data['sentiment'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=72)

#### Logistic Model

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver="lbfgs")

# Create pipeline using Bag of Words
pipe = Pipeline([('vectorizer', data['stopcleaned_review']),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

#### Evaluating Logistic Regression

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print(" test Accuracy:",metrics.accuracy_score(y_test, predicted))
print(" Precision:",metrics.precision_score(y_test, predicted, average=None))
print(" Recall:",metrics.recall_score(y_test, predicted, average=None))

### KNN Classification

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [ ]:
np.random.seed = 10

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform') #here we can change the K-neighbors

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
knn.score(X_test, y_test)

### Decision Trees

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
clf = DecisionTreeClassifier(criterion='entropy')

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
clf.get_depth()

#### Tuning the depth of the tree

In [ ]:
scores = []
for d in range(1, 21):
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=d)
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))

In [ ]:
plt.plot(scores)
plt.ylabel('accuracy', fontsize=15)
plt.xlabel('depth', fontsize=15)

In [ ]:
np.argmax(scores)

 We need to have: 
* Precision & Recall for all methods 
* Precision-Recall curve 
* Cross-validation for all methods 
